In [12]:

from langchain_community.graphs import Neo4jGraph
from langchain_ollama import ChatOllama
import re
from langchain_core.prompts import  PromptTemplate
import uuid
from dotenv import load_dotenv

load_dotenv(override = True)

True

In [13]:
graph = Neo4jGraph()

In [22]:
topic_title = 'Could U.S. involvement in Iran trigger a larger global war?'

In [23]:
original_post = graph.query(
    """
    MATCH (p:OriginalPost)
    WHERE p.topic_title = $topic_title
    RETURN p
    """,
    {'topic_title': topic_title}
)
original_post_id = original_post[0]['p']['id'] if original_post and 'p' in original_post[0] and 'id' in original_post[0]['p'] else None


In [24]:
argument_count_result = graph.query(
    """
    MATCH (a:Argument)--(p)
    WHERE (p:Post OR p:OriginalPost) AND p.topic_title = $topic_title
    RETURN count(DISTINCT a) AS total_arguments
    """,
    {'topic_title': topic_title}
)

total_arguments = argument_count_result[0]['total_arguments'] if argument_count_result else 0
print(f"Total arguments for topic: {total_arguments}")

Total arguments for topic: 23


In [25]:
results = graph.query(
    """
    MATCH (a:Argument)--(mn:MaxNeefCategory), (a)--(p)
    WHERE (p:Post OR p:OriginalPost) AND p.topic_title = $topic_title
    RETURN mn.id AS category, collect(DISTINCT a.motivations_descriptions) AS motivations, count(DISTINCT a) AS argument_count
    ORDER BY category
    """, 
    {'topic_title':topic_title}
)


In [ ]:
MODEL = "deepseek-r1:8b"

summary_extractor = ChatOllama(
        model=MODEL,
        temperature=0,
        #format="json"
    )

summary_extraction_prompt = PromptTemplate(
    input_variables=["motivations"],
    template= """
    Your task is to process the following strands of text, containing the descriptions of various motivations, and summarize it.
    Make a sumarization of the most common themes and what was overall said. 
    You dont need to go too in depth, only enough to get the pig picture.
    The text comes from many users on the internet, on the context of a discussion forum. 
    Input:

    \"\"\"{motivations}\"\"\"
    """
    )
    
summary_chain = summary_extraction_prompt | summary_extractor


In [27]:
def remove_think_tags(text):
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)


In [ ]:
def create_summary_node(new_id, description, n_arguments_used):
    # Ensures that the MotivationSummary node with this ID does not exist before creating it
    presence = n_arguments_used / total_arguments if total_arguments else 0

    graph.query(
        """
        MERGE (n:MotivationSummary {id: $id})
        ON CREATE SET n.description = $description,
                      n.n_arguments_analyzed = $n_arguments_analyzed,
                      n.n_arguments_used = $n_arguments_used,
                      n.presence = $presence
        ON MATCH SET n.description = $description,
                     n.n_arguments_analyzed = $n_arguments_analyzed,
                     n.n_arguments_used = $n_arguments_used,
                     n.presence = $presence
        """,
        {
            "id": new_id,
            "description": description,
            "n_arguments_analyzed": total_arguments,
            "n_arguments_used": n_arguments_used,
            "presence": presence
        }
    )

    return False


def create_summary_relations(new_id, category_id, post_id):
    # Creates a REFLECTS relationship only if it does not already exist
    graph.query(
        """
        MATCH (a:MotivationSummary {id: $novo_id}), (b:MaxNeefCategory {id: $categoria_id})
        MERGE (a)-[:REFLECTS]->(b)
        """,
        {"novo_id": new_id, "categoria_id": category_id}
    )

    # Creates a SUMMARIZES relationship only if it does not already exist
    graph.query(
        """
        MATCH (a:MotivationSummary {id: $novo_id}), (b:OriginalPost {id: $post_id})
        MERGE (a)-[:SUMMARIZES]->(b)
        """,
        {"novo_id": new_id, "post_id": post_id}
    )

    return False

In [ ]:
for line in results:


    # Process text
    print("Category: ", line.get("category"))
    print("Motivations: ", line.get("motivations"))
    print("Arguments present: ", line.get("argument_count"))


    input = {
        #'category' : line.get("category"),
        'motivations' : line.get("motivations")
    }

    response = summary_chain.invoke(input).content
    description = remove_think_tags(response)


    # Create nodes
    id = str(uuid.uuid4())

    create_summary_node(new_id=id, description=description, n_arguments_used=line.get("argument_count"))


    # Create relationships

    create_summary_relations(new_id=id, category_id=line.get("category"), post_id=original_post_id)

    

Category:  Affection
Motivations:  [['Wants to safeguard civilian lives and global stability by opposing military invasions.', 'Seeks to critically analyze and understand patterns of geopolitical aggression and leadership failures.', 'Values national sovereignty and self-determination, opposing violations of freedom by powerful states.', 'Expresses empathy for victims of war and conflict who suffer from aggressive military actions.']]
Argumentos presentes:  1
Category:  Creativity
Motivations:  [['Wants readers to grasp how shifts in national power reshape global strategy and influence.', 'Advocates replacing nostalgic visions with innovative, forward-looking policies to stay relevant.', 'Seeks to motivate public and policy engagement by highlighting the contest over international leadership.']]
Argumentos presentes:  1
Category:  Freedom
Motivations:  [['Wants to maintain regional stability and safety by preventing a nuclear arms race among neighboring countries.', "Seeks to preserve 